<a href="https://colab.research.google.com/github/gabselbach/TCC-implementacoes/blob/master/An%C3%A1lise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>